## Python Imports

In [1]:
import librosa
import spotipy
import os, requests, time, random, json

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

Using TensorFlow backend.


In [2]:
from src.obtain.spotify_metadata import generate_token, download_playlist_metadata

from src.vinyl.build_datasets import extract_features
from src.vinyl.build_datasets import build_dataset

import src.vinyl.db_manager as crates

## [Globals](https://www.geeksforgeeks.org/global-local-variables-python/)

In [3]:
# globals
spotify_username = 'djconxn'
user_id = "spotify:user:djconxn"
zoukables_uri = "spotify:playlist:79QPn32wwghlJfTImywNgV"

zouk_features_path = "data/zoukable_spectral.npy"

## Model Config

### Features Set

In [4]:
features_dict = {
    librosa.feature.mfcc : {'n_mfcc':12},
    librosa.feature.spectral_centroid : {},
    librosa.feature.chroma_stft : {'n_chroma':12},
    librosa.feature.spectral_contrast : {'n_bands':6},
    #librosa.feature.tempogram : {'win_length':192}
}

### Model Architecture
#### TODO: Design a schema for configuring Keras models to build

In [ ]:
# seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
#                    input_shape=input_shape,
#                    padding='same', return_sequences=True))
# seq.add(BatchNormalization())

# seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
#                    padding='same', return_sequences=True))
# seq.add(BatchNormalization())

# seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
#                    padding='same', return_sequences=True))
# seq.add(BatchNormalization())

# seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
#                    padding='same', return_sequences=True))
# seq.add(BatchNormalization())

# seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
#                activation='sigmoid',
#                padding='same', data_format='channels_last'))

# seq.compile(loss='binary_crossentropy', optimizer='adadelta')

# Keras optimizer defaults:
# Adam   : lr=0.001, beta_1=0.9,  beta_2=0.999, epsilon=1e-8, decay=0.
# RMSprop: lr=0.001, rho=0.9,                   epsilon=1e-8, decay=0.
# SGD    : lr=0.01,  momentum=0.,                             decay=0.


# Obtain Data

Set up the Spotify client, download metadata from a Zouk playlist and a non-Zouk playlist.

Download song mp3 samples.

## Authenticate Spotify Client

In [5]:
token=generate_token(username=spotify_username)
sp = spotipy.Spotify(auth=token)

## Download Zouk Playlist Metadata

In [6]:
zouk_songs = crates.download_playlist_songs(sp, user_id, "zoukables", zoukables_uri)
# zouk_metadata = download_playlist_metadata(user_id, zoukables_uri, "pname", sp)

## Download Zouk Playlist Sample mp3's

In [7]:
# zouk_songs = crates.get_playlist_songs('zoukables')
for song_id in zouk_songs:
    crates.get_preview_mp3(song_id)

## Sample Non-Zouk Songs
#### TODO: Remove songs in `zoukables` list

In [8]:
non_zouk_songs = crates.sample_other_songs(n_songs=len(zouk_songs), skip_genres=["zoukables"])

# Calculate Audio Features for Songs



Sample 10 other genres. Add the songs from their playlists to one list. Sample `n_zouk_songs` from that list. Use these as negative cases for training our zouk classifier. Train to convergence, then repeat with another sample of non-zouk songs.

## Build Audio Features

#### TODO: save features to `/data/librosa_features`
Saving the feature array to a numpy file is a terrible caching practice.

New workflow for `build_dataset`:
- Download preview mp3's, extract features and save to `/data/librosa_features`
- Return list of (unique) mp3's successfully downloaded + extracted
- Add new songs if needed for balanced training sets
- Build training dataset from already-extracted features

In [9]:
zouk_data = build_dataset(zouk_songs, features_dict)
non_zouk_data = build_dataset(non_zouk_songs, features_dict)

## Build Targets

In [10]:
target = np.array([1] * len(zouk_songs) + [0] * len(non_zouk_songs))

## Train Test Split

In [11]:
print(zouk_data.shape)
print(non_zouk_data.shape)

(764, 1294, 32)
(764, 1294, 32)


In [12]:
X = np.concatenate((zouk_data, non_zouk_data))

train_idx, test_idx, y_train, y_test = train_test_split(
    range(X.shape[0]), target, test_size=0.33, random_state=42, stratify=target)

X_train = X[train_idx,:,:]
X_test = X[test_idx,:,:]

# Generating Sequences for an LSTM Classifier

## Build Model

#### TODO: Study Convolutional LSTMs
I think this would make the model robust to handling similar songs in different keys

In [13]:
input_shape = (X_train.shape[1], X_train.shape[2])
print("Build LSTM model ...")
model = Sequential()

model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape))
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.35, return_sequences=True))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=1, activation="sigmoid"))

print("Compiling ...")
opt = Adam()
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

Build LSTM model ...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Compiling ...


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1294, 128)         82432     
_________________________________________________________________
lstm_2 (LSTM)                (None, 1294, 64)          49408     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 144,289
Trainable params: 144,289
Non-trainable params: 0
_________________________________________________________________


## Train Model
#### TODO: log the training reports to keep track of learning rates and training times.

In [ ]:
print("Training ...")
batch_size = 35  # num of training examples per minibatch
num_epochs = 400
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs, 
    validation_split=.25, 
    verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=1),
    ]
)

Training ...
Train on 767 samples, validate on 256 samples
Epoch 1/400
767/767 [==============================] - 262s 341ms/step - loss: 0.6681 - acc: 0.5737 - val_loss: 0.6204 - val_acc: 0.6602
Epoch 2/400
767/767 [==============================] - 243s 317ms/step - loss: 0.6135 - acc: 0.6323 - val_loss: 0.5807 - val_acc: 0.6953
Epoch 3/400
767/767 [==============================] - 274s 357ms/step - loss: 0.6006 - acc: 0.6454 - val_loss: 0.5707 - val_acc: 0.7070
Epoch 4/400
767/767 [==============================] - 272s 355ms/step - loss: 0.5910 - acc: 0.6714 - val_loss: 0.5519 - val_acc: 0.7109
Epoch 5/400
767/767 [==============================] - 274s 357ms/step - loss: 0.5808 - acc: 0.6649 - val_loss: 0.5517 - val_acc: 0.6953
Epoch 6/400
767/767 [==============================] - 265s 346ms/step - loss: 0.5528 - acc: 0.6832 - val_loss: 0.5382 - val_acc: 0.7070
Epoch 7/400
767/767 [==============================] - 276s 360ms/step - loss: 0.5435 - acc: 0.7040 - val_loss: 0.5161 

## Evaluate Model

In [ ]:
print("\nTesting ...")
score, accuracy = model.evaluate(
    X_test, y_test, batch_size=batch_size, verbose=1
)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)

## Save Model

In [ ]:
model.save("models/zouk_classifier_spectral_LSTM3.h5")

# Is It Any Good?

Do some explanatory analysis to see what songs are being misclassified. I know that the "labels" are sketchy, so I'll need to do some data cleaning and re-training. How bad is it?

## Get Predictions From Training Set

In [ ]:
all_songs = pd.DataFrame({'song_id':zouk_songs + non_zouk_songs,
                          'target':target})

trainers = all_songs.iloc[train_idx,:].reset_index()

sample0 = trainers[trainers.target==0].sample(10).index
sample1 = trainers[trainers.target==1].sample(10).index
sample_idx = sample0.append(sample1)
samples = trainers.loc[sample_idx]

## Print Classification Report

In [ ]:
y_pred = model.predict(X_train[sample_idx,:])
y_pred_bool = y_pred > 0.75
samples['prediction'] = y_pred_bool.astype(int)
print(classification_report(samples.target, y_pred_bool))

#### TODO: Add False Positives, False Negatives to Spotify playlists

In [ ]:
candidates_uri = 'spotify:playlist:69K5ogTF87NeSFvU9ePI3x'
suspects_uri = 'spotify:playlist:3M1IBVChmAYh7srqwK0CDt'

def update_screening_playlists(false_positives, false_negatives):
    global user_id
    global candidates_uri
    global suspects_uri
    sp.user_playlist_add_tracks(user_id, candidates_uri, false_positives)
    sp.user_playlist_add_tracks(user_id, suspects_uri, false_negatives)

## Sample False Positives and False Negatives

In [ ]:
fp_index = samples[(samples.target==0) & (samples.prediction==1)].index
fn_index = samples[(samples.target==1) & (samples.prediction==0)].index

print("False Positives:")
for i in fp_index:
    song_id = samples['song_id'][i]
    filepath = crates.get_preview_mp3(song_id)
    print(crates.load_song_metadata(song_id)['title'])
    ipd.display(ipd.Audio(filepath))

print("~" * 32)

print("False Negatives:")
for i in fn_index:
    song_id = samples['song_id'][i]
    filepath = crates.get_preview_mp3(song_id)
    print(crates.load_song_metadata(song_id)['title'])
    ipd.display(ipd.Audio(filepath))


# Ship It!

Create a new notebook and copy over the code it needs to run the app from scratch.

Copy over the functions that return the output, and then iterate running the function and copying over the imports and function definitions that are needed to get it to execute without crashing.

(MVP for this should probably run on a single song, not all the songs on a playlist... downloading and extracting the features for many songs is going to take a long time.)

# References

- [Every Noise At Once](http://everynoise.com/)
- [Keras docs](https://keras.io/)
- [Librosa docs](https://librosa.github.io/librosa/index.html)
- [Spotipy docs](https://spotipy.readthedocs.io)
- [ruohoruotsi: LSTM Music Genre Classification on GitHub](https://github.com/ruohoruotsi/LSTM-Music-Genre-Classification)
- [Music Genre classification using a hierarchical Long Short Term Memory (LSTM) Model](http://www.cs.cuhk.hk/~khwong/p186_acm_00_main_lstm_music_rev5.pdf)
- [Using CNNs and RNNs for Music Genre Recognition](https://towardsdatascience.com/using-cnns-and-rnns-for-music-genre-recognition-2435fb2ed6af) [(GitHub)](https://github.com/priya-dwivedi/Music_Genre_Classification)
- [The dummy’s guide to MFCC](https://medium.com/prathena/the-dummys-guide-to-mfcc-aceab2450fd)
- [Convolutional LSTM Network: A Machine Learning Approach for Precipitation Nowcasting](https://arxiv.org/abs/1506.04214v1)
- [An introduction to ConvLSTM](https://medium.com/neuronio/an-introduction-to-convlstm-55c9025563a7)

# Storage Space Requirements

.model files = 1 - 6 MB

features = 250MB (spectral), 1.7GB(tempo)

mp3 previews = 365 kB ea

librosa features = 420 kB ea

# Action Plan

This process should train a decent classifier for songs from this playlist, but I really need to find a much larger list of positive cases. My plan is to maintain three playlists on Spotify: 
- the Zoukables list, which I've curated
- a False Positives list, non-zouk songs which have been classified as zoukable and may very well be zoukable (since in our workflow, "negative" just means "has not been tagged positive"), which I can then screen and possibly add to the Zoukables list
- a False Negatives list, zouk songs which have been classified as not zoukable and may not actually belong in the Zoukables list

Once I set up these playlists and connect them to my pipeline, I can run and re-run the training pipeline, and listen and screen the Spotify playlists to curate my training set.

And then the next step would be to engineer a system where other users can vote on songs to add to the Zoukables list, and automatically add songs with a threshold of votes and a high enough percentage of Yes votes.

## Spotify Playlist Updates

- Refresh Zoukables list when training models
- Update FP/FN screening playlists on Spotify
- Update GitHub

## Re-implement EveryNoise Scraper

(I think this is working)

- Download EveryNoise playlist URLs
- Download Spotify playlist metadata
- Download preview mp3s (during model training)
- Update GitHub

## Mongo DB: Songs Database

(I've got this working in flat files)

- Song IDs
- Spotify metadata
- Librosa Features
- Genre Labels
- Python API (1.4.0.1/2/3, 2.1.0.1)
- Update GitHub

## Mongo DB: Models Database

- Keras schema (0.3.2.1)
- Feature sets
- Training reports
- Python API (3.1.0.1, 3.2.0.1)
- Update GitHub

## Python Package

- Keras model API
- Organize modules
- Write docstrings
- Conda environment
- Update GitHub

## Deployment
- Reproduce pipeline on other machines
- Reproduce pipeline for other genres
- Deploy to AWS
